In [1]:
import os
import praw
from CONFIG import SUBMISSION_DATA_FIELDS

import pandas as pd

from dotenv import find_dotenv, load_dotenv

In [10]:
load_dotenv(find_dotenv())

CLIENT_ID=os.getenv("CLIENT_ID")
SECRET_TOKEN=os.getenv("SECRET_TOKEN")
USERNAME=os.getenv("USERNAME")
PASSWORD=os.getenv("PASSWORD")

reddit = praw.Reddit(client_id=CLIENT_ID,
                     client_secret=SECRET_TOKEN,
                     password=PASSWORD,
                     username=USERNAME,
                     user_agent='Tutorial')

print(reddit.user.me())

andreaLolli


In [14]:

# Function to search for a query and list subreddits
def search_and_list_subreddits(query, limit=10):
    # Search Reddit for the query
    search_results = reddit.subreddit('all').search(query, limit=limit)
    
    # Collect the subreddits from the search results
    subreddits = [result.subreddit.display_name for result in search_results]

    # Print the subreddits
    print(f"Top {limit} subreddits for the query '{query}':")
    return subreddits

In [18]:
subreddits_names = search_and_list_subreddits(query="Drake AND Kendrick Lamar")

Top 10 subreddits for the query 'Drake AND Kendrick Lamar':


In [25]:
def fetch_submission_data(subreddit_name, keywords, fields=SUBMISSION_DATA_FIELDS, limit=100):
    """
    Fetches submission data from a given subreddit based on search keywords.

    Args:
        subreddit_name (str): The name of the subreddit to search in.
        keywords (str): The search keywords.
        limit (int): The maximum number of search results to fetch.

    Returns:
        pd.DataFrame: A DataFrame containing submission data.
    """
    submissions_data = []

    for submission in reddit.subreddit(subreddit_name).search(keywords, limit=limit):
        submission_data = {}

        for field in fields:
            try:
                submission_data[field] = getattr(submission, field, '')
            except AttributeError:
                submission_data[field] = ''

        submissions_data.append(submission_data)    

    return pd.DataFrame(submissions_data)

NameError: name 'SUBMISSION_DATA_FIELDS' is not defined

In [26]:
submissions_data = fetch_submission_data(
    subreddit_name=subreddits_names[0],
    keywords="Kendrick Lamar AND Drake",
    limit=10
    )

submissions_data.head()

NameError: name 'SUBMISSION_DATA_FIELDS' is not defined